# Creditworthiness Research


## Data Loading and Exploration

In [79]:
import os
import pandas as pd

In [80]:
BASE_DIR = os.getcwd()
data = pd.read_csv(f'{BASE_DIR}/datasets/data.csv')

In [81]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,higher,0,Married,0,F,Employee,0,253875.639453,Purchase of housing
1,1,-4024.803754,36,middle,1,Married,0,F,Employee,0,112080.014102,Car purchase
2,0,-5623.422610,33,Middle,1,Married,0,M,Employee,0,145885.952297,Purchase of housing
3,3,-4124.747207,32,middle,1,Married,0,M,Employee,0,267628.550329,Additional education
4,0,340266.072047,53,middle,1,Civil partnership,1,F,Pensioner,0,158616.077870,Wedding
5,0,-926.185831,27,higher,0,Civil partnership,1,M,Partner,0,255763.565419,Purchase of housing
6,0,-2879.202052,43,higher,0,Married,0,F,Partner,0,240525.971920,Housing transactions
7,0,-152.779569,50,MIDDLE,1,Married,0,M,Employee,0,135823.934197,Education
8,2,-6929.865299,35,HIGHER,0,Civil partnership,1,F,Employee,0,95856.832424,Wedding expenses
9,0,-2188.756445,41,middle,1,Married,0,M,Employee,0,144425.938277,Purchase of housing for the family


In [82]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [83]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,19351.0,167422.302208,102971.566448,20667.263793,103053.152913,145017.937533,203435.067663,2.265604e+06


The table contains a total of 12 columns with different data types.  
  
`children` - the number of children in the family.  
`days_employed` - total days of employment.  
`dob_years` - client's age in years.  
`education` - client's education level.  
`education_id` - identifier of education level.  
`family_status` - marital status.  
`family_status_id` - identifier of marital status.  
`gender` - client's gender.  
`income_type` - employment type.  
`debt` - whether the client had a loan default.  
`total_income` - monthly income.  
`purpose` - purpose of the loan.

### **Conclusions**

  
Each row in the table contains information about a specific individual applying for credit for a known purpose. From the table, we can observe the marital status, number of children, loan repayment, and employment history.   
The `children` column has values ranging from -1 to 20.  
The `days_employed` column has negative values, and the average value is `63046.497661` days, which needs further investigation. The minimum value of the `dob_years` feature is 0, which also needs to be corrected.

TO DO:  
- Check for missing values and duplicates.  
- Check the `children` column for artifacts and incorrect values.  
- Check for negative values in the `days_employed` column and identify the reason for this phenomenon.  
- Correct the age value of 0.  
- Convert the `education` and `family_status` columns to lowercase.  
- Downcast data types.

## Data Preprocessing

### Handling Missing Values

In [84]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Two columns have missing values: `days_employed` and `total_income`. We will address the missing values in the `days_employed` column later. First, let's handle the missing values in the `total_income` column. Since the income type has the most significant impact on the income amount, we can fill the missing values in this column with the median value for each income type from the `income_type` column.

In [85]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Handling Anomalous Values

In [86]:
data[data['days_employed'] < 0]['days_employed'].count()

15906

The `days_employed` column contains many negative values, which contradicts the nature of the data in the column. Let's replace them with positive values.

In [87]:
data['days_employed'] = data['days_employed'].abs()

Let's examine the employment period values (`days_employed`) by income type.

In [88]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
Civil servant           2689.368353
Employee                1574.202821
Entrepreneur             520.848083
On maternity leave      3296.759962
Partner                 1547.382223
Pensioner             365213.306266
Student                  578.751554
Unemployed            366413.652744
Name: days_employed, dtype: float64

Two income types (unemployed and retirees) have unusually large values. There is no obvious approach to correcting such values, so we will leave them as they are.

### Handling Outliers in the `children` Column

In [89]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

The value -1 is clearly an error, and 20 is highly unlikely. Since the proportion of anomalous values is small, we will simply remove these entries.

In [90]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [91]:
data['children'].value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Let's check the result.

### Handling Missing Values (Again)

We will fill the missing values in the `days_employed` column with the median values for each income type (`income_type`)

In [92]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Let's check the dataset again for any remaining missing values.

In [93]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Data Type Conversion

We will convert the `total_income` column from `float` to `int`.

In [94]:
data['total_income'] = data['total_income'].astype(int)

### Handling Duplicates

In [95]:
data['education'].unique()

array(['higher', 'middle', 'Middle', 'MIDDLE', 'HIGHER',
       'incomplete higher', 'elementary', 'Higher', 'INCOMPLETE HIGHER',
       'Incomplete higher', 'ELEMENTARY', 'Elementary', 'Academic degree',
       'ACADEMIC DEGREE', 'academic degree'], dtype=object)

The `education` column contains the same values but written in different cases: uppercase and lowercase. Let's convert them to lowercase.

In [96]:
data['education'] = data['education'].str.lower()

Let's check for complete duplicates.

In [97]:
data.duplicated().sum()

79

Let's remove them.

In [98]:
data = data.drop_duplicates()

### Data Categorization

The task at hand is as follows:

To answer whether marital status and the number of children have an impact on loan repayment. We will categorize the data as follows: 

- Based on income (`total_income`):  
    - `E` - < 30,000;
    - `D` - 30,001 to 50,000;
    - `C` - 50,001 to 200,000;
    - `B` - 200,001 to 1,000,000;
    - `A` - > 1,000,000    
   
    
      
- Based on the loan purpose (existing categories need to be consolidated);

Let's start with the first point and categorize the data based on income.  
We will use our custom function `categorize_income()` and the `apply()` method.

In [99]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [100]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

We will consolidate the loan purposes into broader categories.

In [101]:
data['purpose'].unique()

array(['Purchase of housing', 'Car purchase', 'Additional education',
       'Wedding', 'Housing transactions', 'Education', 'Wedding expenses',
       'Purchase of housing for the family', 'Real estate purchase',
       'Purchase of commercial real estate',
       'Purchase of residential real estate',
       'Construction of own real estate', 'Real estate',
       'Construction of real estate', 'Purchase of a used car',
       'Purchase of own car', 'Transactions with commercial real estate',
       'Construction of residential real estate', 'Housing',
       'Transactions with own real estate', 'Cars',
       'Engagement in education', 'Transaction with a used car',
       'Acquisition of education', 'Car',
       'Obtaining additional education', 'Purchase of own housing',
       'Transactions with real estate', 'Obtaining higher education',
       'Own car', 'Car transaction', 'Specialized education',
       'Higher education', 'Purchase of housing for rental',
       'Housing rep

All purposes can be divided into 4 groups:

- `'car operations'`,
- `'real estate operations'`,
- `'wedding'`,
- `'education'`.

Let's write a function `categorize_purpose()` and use the `apply()` method to add a category feature to the dataset.

In [102]:
def categorize_purpose(row):
    try:
        if 'car' in row:
            return 'car'
        elif 'house' in row or 'estate' in row:
            return 'housing'
        elif 'wedd' in row:
            return 'wedding'
        elif 'educ' in row:
            return 'education'
    except:
        return 'without category'

In [103]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Answering Questions about the Data

#### Is there a correlation between the number of children and loan repayment? 

In [104]:
data.groupby('children')['debt'].agg(['count','mean'])

,count,mean
children,,
0,14084,0.075476
1,4808,0.092346
2,2051,0.094588
3,330,0.081818
4,41,0.097561
5,9,0.000000


In [105]:
data['parent'] = data['children'] > 0
data.groupby('parent')['debt'].agg(['count','mean'])

,count,mean
parent,,
False,14084,0.075476
True,7239,0.092416


Borrowers with children are more likely to have loan defaults compared to borrowers without children. However, there is no clear correlation based on the number of children, but rather on their presence or absence.

#### Is there a correlation between marital status and loan repayment? 

In [106]:
data.groupby('family_status')['debt'].agg(['count','mean']).sort_values(by = 'mean')

,count,mean
family_status,,
Widow / Widower,951,0.066246
Divorced,1189,0.070648
Married,12261,0.075606
Civil partnership,4126,0.093311
Single,2796,0.097639


There is a correlation between marital status and loan repayment. Widows and widowers have the lowest percentage of loan defaults, while unmarried individuals have the highest percentage.

#### Is there a correlation between income level and loan repayment? 

In [107]:
data.groupby('total_income_category')['debt'].agg(['count','mean'])

,count,mean
total_income_category,,
A,25,0.080000
B,5014,0.070602
C,15913,0.085025
D,349,0.060172
E,22,0.090909


The number of borrowers varies significantly across different income groups. In the categories with the lowest and highest incomes, there is insufficient data to draw any conclusions, and the same applies to category D. When comparing categories B and C (the second and third positions in terms of income), there is a correlation between income level and loan repayment.

#### How do different loan purposes affect loan repayment? 

In [108]:
data.groupby('purpose_category')['debt'].agg(['count','mean']).sort_values(by = 'mean')

,count,mean
purpose_category,,
housing,6314,0.074755
car,1929,0.092794
education,3546,0.095037


The repayment percentage varies depending on the loan purpose. Borrowers with real estate loans have the highest repayment percentage, while borrowers with car loans have the highest default rate.

### Conclusion

As a result of the creditworthiness research, the following correlations were identified:

- Among the examined indicators, there is no clear correlation based on income. It is necessary to either obtain more data on borrowers with the highest and lowest incomes or redefine the boundaries of income categories.  
  
- Loan repayment is influenced by the presence or absence of children, rather than their quantity.  
  
- Successful loan repayment also depends on the loan purpose and the borrower's marital status.  
  
**Let's try to create a profile of an ideal borrower**:

An ideal borrower is a widower without children, with above-average income, taking a loan for real estate.